# Install Dependencies

In [ ]:
!pip install git+https://github.com/microsoft/dowhy.git econml==0.12.0 pandas==1.3.0 xgboost==1.4.2 numba==0.53.1 numpy==1.21.0 graphviz==0.17 matplotlib==3.4.2

In [ ]:
import pandas as pd

# Load Data

In [ ]:
output = pd.read_csv('./output.csv', sep=',', compression=None)
output.head()

# Estimate Effects

## Additional Code

In [ ]:
#
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project.
#

import itertools

from typing import Dict, List

import networkx as nx


class CausalGraph:
    """
    Generate gml-formatted causal graphs given treatment, outcome,
    confounder and effect modifier specifications.
    This simplifies the json input by enabling front end to pass on a list
    of confounders and effect modifiers as model specs instead of passing
    on actual gml graphs.
    """

    def __init__(
        self,
        treatment_specs: List[Dict],
        outcome_specs: List[Dict],
        model_specs: List[Dict],
    ):
        self.treatment_specs = treatment_specs
        self.outcome_specs = outcome_specs
        self.model_specs = model_specs

    def create_gml_graph(self, model_spec) -> List:
        """
        Given input for a model type (e.g. maximum model),
        create associated causal graphs in the gml format
        """
        confounders = model_spec["confounders"]
        effect_modifiers = model_spec["outcome_determinants"]
        model_type = model_spec["type"]
        model_label = model_spec["label"]

        graphs = []
        for treatment_outcome in list(
            itertools.product(self.treatment_specs, self.outcome_specs)
        ):
            treatment = treatment_outcome[0]["variable"]
            outcome = treatment_outcome[1]["variable"]
            causal_graph = nx.DiGraph()

            # add nodes
            for variable in [treatment] + [outcome] + confounders + effect_modifiers:
                causal_graph.add_node(variable)

            # add confounder edges
            for confounder in confounders:
                causal_graph.add_edge(confounder, treatment)
                causal_graph.add_edge(confounder, outcome)

            for modifier in effect_modifiers:
                causal_graph.add_edge(modifier, outcome)

            # add treatments to outcome edges
            causal_graph.add_edge(treatment, outcome)

            graphs.append(
                {
                    "type": model_type,
                    "label": model_label,
                    "treatment": treatment,
                    "outcome": outcome,
                    "confounders": confounders,
                    "effect_modifiers": effect_modifiers,
                    "causal_graph": " ".join(nx.generate_gml(causal_graph)),
                }
            )
        return graphs

    def create_gml_model_specs(self) -> List:
        """
        Generate gml causal graphs for all model types in the model specs
        (e.g. maximum, intermediate, minimum, and unadjusted models)
        """
        graphs = []
        for spec in self.model_specs:
            graphs = graphs + self.create_gml_graph(spec)
        return graphs


In [ ]:
#
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project.
#

import os


DEFAULT_REFUTATION_TESTS = [
    "random_common_cause",
    "placebo_treatment_refuter",
    "data_subset_refuter",
    "bootstrap_refuter",
]

# Failing these refuters should only result in warning,
# not automatic rejection of the estimate
SENSITIVITY_REFUTERS = ["add_unobserved_common_cause"]

# whether to include sensitivity test
INCLUDE_SENSITIVITY_REFUTERS = False

# default number of simulations used for refutation
DEFAULT_REFUTATION_SIMULATIONS = 100

# default number of simulations used for bootstrap-based confidence intervals
DEFAULT_CI_SIMULATIONS = 100

# default number of simlations used for significance test computation
DEFAULT_SIGNIFICANCE_TEST_SIMULATIONS = 100


def get_batch_size(env_var):
    batch_size = os.environ.get(env_var, None)
    if batch_size is not None:
        return int(batch_size)
    else:
        return None


def get_batch(iterable, size):
    if size is None:
        size = len(iterable)
    length = len(iterable)
    for start in range(0, length, size):
        end = min(start + size, length)
        yield (length, end, iterable[start:end])


In [ ]:
#
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project.
#

import copy

from typing import Dict

from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from xgboost import XGBClassifier, XGBRegressor


class CausalEstimator:
    """
    Create estimator configurations for a given estimator method
    using a format expected by DoWhy
    """

    def __init__(self, classifier=None, regressor=None, parallelism=False):
        """
        Create default xgboost classifier and regressor models
        that could be used at different stages of an estimator.
        Using grid search to auto tune hyperparameters for these models.
        """
        self.parallelism = parallelism

        if classifier:
            self.model_classifier = classifier
        else:
            # classifier params
            self.classifier_kwargs = {
                "learning_rate": 0.1,
                "n_estimators": 100,
                "objective": "binary:logistic",
                "nthread": -1 if parallelism else 1,
                "use_label_encoder": False,
                "random_state": 42,
                "verbosity": 0,
            }
            self.default_model_classifier = XGBClassifier(
                max_depth=3, **self.classifier_kwargs
            )
            self.model_classifier = GridSearchCV(
                estimator=XGBClassifier(**self.classifier_kwargs),
                param_grid={"max_depth": [3, 5], "colsample_bytree": [0.5, 0.8, 1.0]},
                cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
                n_jobs=1,
                scoring="neg_log_loss",
                refit=False,
                verbose=0,
            )

        if regressor:
            self.model_regressor = regressor
        else:
            # regressor params
            self.regressor_kwargs = {
                "learning_rate": 0.1,
                "n_estimators": 100,
                "objective": "reg:squarederror",
                "nthread": -1 if parallelism else 1,
                "random_state": 42,
                "verbosity": 0,
            }

            self.default_model_regressor = XGBRegressor(
                max_depth=3, **self.regressor_kwargs
            )

            self.model_regressor = GridSearchCV(
                estimator=XGBRegressor(**self.regressor_kwargs),
                param_grid={"max_depth": [3, 5], "colsample_bytree": [0.5, 0.8, 1.0]},
                cv=KFold(n_splits=3, shuffle=True, random_state=42),
                n_jobs=1,
                scoring="neg_mean_squared_error",
                refit=False,
                verbose=0,
            )

        self.estimator_configurators = {
            "backdoor.econml.dr.ForestDRLearner": "config_forest_doubly_robust",
            "backdoor.econml.dr.LinearDRLearner": "config_doubly_robust",
            "backdoor.econml.dml.CausalForestDML": "config_forest_double_machine_learning",
            "backdoor.econml.dml.LinearDML": "config_double_machine_learning",
            "backdoor.propensity_score_weighting": "config_propensity_weighting",
            "backdoor.propensity_score_matching": "config_propensity_matching",
            "backdoor.propensity_score_stratification": "config_propensity_stratification",
            "backdoor.linear_regression": "config_default",
        }

    def config_estimator(self, estimator_spec: Dict) -> Dict:
        config_method = self.estimator_configurators.get(estimator_spec["method_name"])
        if config_method is None:
            return estimator_spec
        else:
            return getattr(self, config_method)(estimator_spec)

    def config_doubly_robust(self, estimator_spec: Dict) -> Dict:
        """
        Config parameters for Linear/Forest Doubly Robust Learner
        """

        estimator_spec["method_params"] = {
            "init_params": {
                "model_propensity": self.model_classifier,
                "model_regression": self.model_regressor,
                "cv": KFold(n_splits=3, shuffle=True, random_state=42),
                "random_state": 42,
            },
            "fit_params": {},
        }
        return estimator_spec

    def config_double_machine_learning(self, estimator_spec: Dict) -> Dict:
        """
        Config parameters for Linear/Forest Double Machine Learning
        """
        estimator_spec["method_params"] = {
            "init_params": {
                "model_t": self.model_classifier,
                "model_y": self.model_regressor,
                "discrete_treatment": True,
                "cv": KFold(n_splits=3, shuffle=True, random_state=42),
                "random_state": 42,
            },
            "fit_params": {},
        }
        return estimator_spec

    def config_forest_double_machine_learning(self, estimator_spec: Dict) -> Dict:
        estimator_spec = self.config_double_machine_learning(estimator_spec)
        estimator_spec["method_params"]["init_params"]["n_jobs"] = (
            -1 if self.parallelism else None
        )
        return estimator_spec

    def config_forest_doubly_robust(self, estimator_spec: Dict) -> Dict:
        estimator_spec = self.config_doubly_robust(estimator_spec)
        estimator_spec["method_params"]["init_params"]["n_jobs"] = (
            -1 if self.parallelism else None
        )
        return estimator_spec

    def config_propensity_weighting(self, estimator_spec: Dict) -> Dict:
        """
        Config parameters for Inverse Propensity Weighting
        """
        estimator_spec["method_params"] = {
            "propensity_score_model": self.model_classifier,
            "recalculate_propensity_score": True,
            "propensity_score_column": "propensity_score",
            "weighting_scheme": "ips_weight",
            "min_ps_score": 0.05,
            "max_ps_score": 0.95,
        }
        return estimator_spec

    def config_propensity_stratification(self, estimator_spec: Dict) -> Dict:
        """
        Config parameters for Propensity Score Stratification
        Number of strata is auto-selected within DoWhy
        """
        estimator_spec["method_params"] = {
            "recalculate_propensity_score": True,
            "propensity_score_model": self.model_classifier,
            "propensity_score_column": "propensity_score",
            "num_strata": "auto",
            "clipping_threshold": 5,
        }
        return estimator_spec

    def config_propensity_matching(self, estimator_spec: Dict) -> Dict:
        """
        Config parameters for Propensity Score Matching
        """
        estimator_spec["method_params"] = {
            "recalculate_propensity_score": True,
            "propensity_score_model": self.model_classifier,
        }

        return estimator_spec

    def config_default(self, estimator_spec: Dict) -> Dict:
        """
        Default setting with no configurations
        """
        estimator_spec["method_params"] = {}
        return estimator_spec

    def tune_classifier_model(self, identified_estimand, causal_model):
        """
        Tune hyperparameters for propensity model
        """
        observed_common_causes = copy.deepcopy(
            identified_estimand.get_backdoor_variables()
        )
        if len(observed_common_causes) > 0:
            treatment_var = identified_estimand.treatment_variable
            variables = observed_common_causes + treatment_var
            training_data = causal_model._data[variables].drop_duplicates()
            best_params = self.model_classifier.fit(
                training_data[observed_common_causes], training_data[treatment_var]
            ).best_params_
            best_model = XGBClassifier(**best_params, **self.classifier_kwargs)
            return best_model
        else:
            return self.default_model_classifier

    def tune_dml_regressor_model(self, identified_estimand, causal_model):
        """
        Tune hyperparameters for first stage regressor model for Double ML model
        """
        observed_common_causes = copy.deepcopy(
            identified_estimand.get_backdoor_variables()
        )
        if len(observed_common_causes) > 0:
            outcome_var = identified_estimand.outcome_variable
            variables = observed_common_causes + outcome_var
            training_data = causal_model._data[variables].drop_duplicates()
            best_params = self.model_regressor.fit(
                training_data[observed_common_causes], training_data[outcome_var]
            ).best_params_
            best_model = XGBRegressor(**best_params, **self.regressor_kwargs)
            return best_model
        else:
            return self.default_model_regressor

    def tune_dr_regressor_model(self, identified_estimand, causal_model):
        """
        Tune hyperparameters for first stage regressor model for doubly robust learner
        """
        observed_common_causes = copy.deepcopy(
            identified_estimand.get_backdoor_variables()
        )
        if len(observed_common_causes) > 0:
            treatment_var = identified_estimand.treatment_variable
            outcome_var = identified_estimand.outcome_variable
            features = observed_common_causes + treatment_var
            variables = features + outcome_var
            training_data = causal_model._data[variables].drop_duplicates()
            best_params = self.model_regressor.fit(
                training_data[features], training_data[outcome_var]
            ).best_params_
            best_model = XGBRegressor(**best_params, **self.regressor_kwargs)
            return best_model
        else:
            return self.default_model_regressor


In [ ]:
#
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project.
#

from math import sqrt
from typing import Dict

import numpy as np

from dowhy.causal_estimator import CausalEstimate
from dowhy.causal_identifier import IdentifiedEstimand
from dowhy.causal_model import CausalModel

from showwhy_inference.inference_config import (
    DEFAULT_REFUTATION_SIMULATIONS,
    DEFAULT_REFUTATION_TESTS,
    INCLUDE_SENSITIVITY_REFUTERS,
    SENSITIVITY_REFUTERS,
)


class Refutation:
    def __init__(self, refuter_spec: Dict):
        self.refuter_spec = refuter_spec

        # placeholder dict so we will still have these columns in the
        # final results in case the refuter can't be run
        self.default_results = {}
        refutation_tests = (
            DEFAULT_REFUTATION_TESTS
            if not INCLUDE_SENSITIVITY_REFUTERS
            else DEFAULT_REFUTATION_TESTS + SENSITIVITY_REFUTERS
        )
        for test in refutation_tests:
            self.default_results[f'refuter_{test.replace("_refuter", "")}'] = None

    def refute_estimate(
        self,
        model: CausalModel,
        identified_estimand: IdentifiedEstimand,
        estimate: CausalEstimate,
    ) -> Dict:
        """
        Run estimate through a given refutation test defined in the refuter_spec dict
        """
        if not self.refuter_spec:
            return self.default_results

        results = {}
        refuter = self.refuter_spec

        if (model is None) or (identified_estimand is None) or (estimate is None):
            results[f'refuter_{refuter["method_name"].replace("_refuter", "")}'] = None
            return results
        else:
            params = [model, identified_estimand, estimate, refuter["num_simulations"]]
            try:
                results[
                    f'refuter_{refuter["method_name"].replace("_refuter", "")}'
                ] = getattr(self, refuter["method_name"])(*params)
            except Exception:
                results[
                    f'refuter_{refuter["method_name"].replace("_refuter", "")}'
                ] = None
            finally:
                return results

    def add_unobserved_common_cause(
        self,
        model: CausalModel,
        identified_estimand: IdentifiedEstimand,
        estimate: CausalEstimate,
        num_simulations: int = DEFAULT_REFUTATION_SIMULATIONS,
        **kwargs,
    ) -> int:
        """
        Simulate a common cause that is correlated with the treatment and outcome.
        The test fails if the new estimate changes sign
            compared to the original estimate
        This test requires domain knowledge to set plausible effect strengths for
            simulating unobserved confounders
        """
        # infer if treatment and outcome are binary variables
        if model._data[model._treatment[0]].nunique() == 2:
            treatment_type = "binary_flip"
        else:
            treatment_type = "linear"
        if model._data[model._outcome[0]].nunique() == 2:
            outcome_type = "binary_flip"
        else:
            outcome_type = "linear"

        num_effect_points = int(round(sqrt(num_simulations), 0))
        if "effect_strength_on_treatment" in kwargs:
            effect_on_treatment = kwargs["effect_strength_on_treatment"]
        else:
            effect_on_treatment = np.linspace(0.0001, 0.01, num_effect_points).tolist()
        if "effect_strength_on_outcome" in kwargs:
            effect_on_outcome = kwargs["effect_strength_on_treatment"]
        else:
            effect_on_outcome = np.linspace(0.0001, 0.01, num_effect_points).tolist()

        refuter = model.refute_estimate(
            identified_estimand,
            estimate,
            method_name="add_unobserved_common_cause",
            confounders_effect_on_treatment=treatment_type,
            confounders_effect_on_outcome=outcome_type,
            effect_strength_on_treatment=effect_on_treatment,
            effect_strength_on_outcome=effect_on_outcome,
        )
        return self.check_sign_change(refuter.new_effect, estimate.value)

    def random_common_cause(
        self,
        model: CausalModel,
        identified_estimand: IdentifiedEstimand,
        estimate: CausalEstimate,
        num_simulations: int = DEFAULT_REFUTATION_SIMULATIONS,
        **kwargs,
    ) -> int:
        """
        Add white noise variable as a random common cause.
        Use p-value as pass/fail criterion.
        """
        refuter = model.refute_estimate(
            identified_estimand,
            estimate,
            method_name="random_common_cause",
            num_simulations=num_simulations,
        )
        return self.check_p_value(refuter.refutation_result["p_value"])

    def placebo_treatment_refuter(
        self,
        model: CausalModel,
        identified_estimand: IdentifiedEstimand,
        estimate: CausalEstimate,
        num_simulations: int = DEFAULT_REFUTATION_SIMULATIONS,
        **kwargs,
    ) -> int:
        """
        Replace treatment with a random independent variable.
        Use p-value as pass/fail criterion.
        """
        refuter = model.refute_estimate(
            identified_estimand,
            estimate,
            method_name="placebo_treatment_refuter",
            placebo_type="permute",
            num_simulations=num_simulations,
        )
        return self.check_p_value(refuter.refutation_result["p_value"])

    def data_subset_refuter(
        self,
        model: CausalModel,
        identified_estimand: IdentifiedEstimand,
        estimate: CausalEstimate,
        num_simulations: int = DEFAULT_REFUTATION_SIMULATIONS,
        **kwargs,
    ) -> int:
        """
        Replace the given subset with a randomly selected subset.
        Use p-value as pass/fail criterion.
        """
        refuter = model.refute_estimate(
            identified_estimand,
            estimate,
            method_name="data_subset_refuter",
            subset_fraction=0.9,
            num_simulations=num_simulations,
        )
        return self.check_p_value(refuter.refutation_result["p_value"])

    def bootstrap_refuter(
        self,
        model: CausalModel,
        identified_estimand: IdentifiedEstimand,
        estimate: CausalEstimate,
        num_simulations: int = DEFAULT_REFUTATION_SIMULATIONS,
        **kwargs,
    ) -> int:
        """
        Replace the given dataset with a bootstrapped sample of the dataset.
        Use p-value as pass/fail criterion.
        """
        refuter = model.refute_estimate(
            identified_estimand,
            estimate,
            method_name="bootstrap_refuter",
            num_simulations=num_simulations,
        )

        return self.check_p_value(refuter.refutation_result["p_value"])

    def check_sign_change(self, new_effect, original_estimate) -> int:
        """
        Helper function to co check if the new effects change sign
            (e.g., negative -> positive or vice versal)
        compared to the original effect
        """
        if not isinstance(new_effect, (list, tuple, np.ndarray)):
            return int(new_effect * original_estimate >= 0)
        else:
            return int(
                new_effect[0] * original_estimate >= 0
                and new_effect[1] * original_estimate >= 0
            )

    def check_p_value(self, p_value: float, p_threshold: float = 0.05) -> int:
        """
        Helper function to check if the returned p_value passes the given threshold
        """
        return int(p_value > p_threshold)


In [ ]:
#
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project.
#

import logging

from typing import List

import lightgbm as lgb
import pandas as pd
import shap

from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error


class SpecificationIntepreter:
    """
    Estimate the contribution of each specification feature to the estimated effect
    using SHAP intepreter
    """

    def __init__(
        self,
        spec_results: pd.DataFrame,
        spec_features: List,
        estimated_effect_col: str = "estimated_effect",
    ):
        self.spec_results = spec_results
        self.spec_features = spec_features
        self.estimated_effect_col = estimated_effect_col

    def interpret(self) -> pd.DataFrame:
        # encode categorical spec features
        encoded_features = []
        le = preprocessing.LabelEncoder()
        for feature in self.spec_features:
            self.spec_results[feature] = self.spec_results[feature].astype(str)
            self.spec_results[f"{feature}_encoded"] = le.fit_transform(
                self.spec_results[feature]
            )
            encoded_features.append(f"{feature}_encoded")

        # create training data
        X = self.spec_results[encoded_features]
        y = self.spec_results[self.estimated_effect_col]
        train_data = lgb.Dataset(X, label=y, categorical_feature=encoded_features)

        # train a LightGBM regressor to predict estimated effects given spec features
        params = {
            "objective": "regression",
            "boosting_type": "gbdt",
            "max_depth": 3,
            "num_leaves": 5,
            "min_data_in_leaf": 1,
            "min_data_per_group": 1,
            "cat_smooth": 1,
        }
        spec_regressor = lgb.train(
            params=params, train_set=train_data, categorical_feature=encoded_features
        )

        # train performance
        y_hat = spec_regressor.predict(X)
        logging.info("Train MAE: {:.3f}".format(mean_absolute_error(y, y_hat)))

        # use SHAP for feature importance
        explainer = shap.TreeExplainer(spec_regressor)
        shap_values = explainer.shap_values(X)

        # save SHAP value for each spec feature
        for i, feature in enumerate(self.spec_features):
            self.spec_results[f"shap_{feature}"] = shap_values[:, i]

        # remove encoded feature columns from the final results
        cleaned_columns = [
            col for col in self.spec_results.columns if col not in encoded_features
        ]
        return self.spec_results[cleaned_columns]


def run_interpreter(results_df: pd.DataFrame) -> pd.DataFrame:
    spec_features = [
        "population_name",
        "treatment",
        "outcome",
        "causal_model",
        "estimator",
    ]
    spec_interpreter = SpecificationIntepreter(results_df, spec_features)
    return spec_interpreter.interpret()


In [ ]:
#
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project.
#

import copy
import itertools
import logging
import time
import warnings

from typing import Dict, List, Tuple

import dowhy
import numpy as np
import pandas as pd

from showwhy_inference.causal_graph import CausalGraph
from showwhy_inference.estimator import CausalEstimator
from showwhy_inference.inference_config import (
    DEFAULT_REFUTATION_TESTS,
    INCLUDE_SENSITIVITY_REFUTERS,
    SENSITIVITY_REFUTERS,
)


warnings.simplefilter("ignore")


def is_valid_spec(spec: List) -> bool:
    """
    Helper function to filter out obviously invalid specs
    """
    _, treatment_spec, outcome_spec, model_spec, estimator_spec = spec

    if (
        (treatment_spec["variable"] != model_spec["treatment"])
        or (outcome_spec["variable"] != model_spec["outcome"])
        or (
            not model_spec["confounders"]
            and not model_spec["effect_modifiers"]
            and estimator_spec["require_propensity_score"]
        )
    ):
        return False
    else:
        return True


def generate_all_specs(
    population_specs, treatment_specs, outcome_specs, model_specs, estimator_specs
):
    """
    Generate all combinations of population, treatment,
    outcome, causal model and estimator
    """

    causal_graph = CausalGraph(treatment_specs, outcome_specs, model_specs)
    model_specs = causal_graph.create_gml_model_specs()

    specs = itertools.product(
        population_specs, treatment_specs, outcome_specs, model_specs, estimator_specs
    )

    return [spec for spec in specs if is_valid_spec(spec)]


def estimate_specification(spec: Tuple, context: Dict) -> Dict:
    start_time = time.time()
    causal_model, identified_estimand, estimate, population_size = __estimate_effect(
        population_spec=spec[0],
        treatment_spec=spec[1],
        outcome_spec=spec[2],
        model_spec=spec[3],
        estimator_spec=spec[4],
        context=context,
    )
    result_dict = {
        "population_type": spec[0]["type"],
        "population_name": spec[0]["label"],
        "population_size": population_size,
        "treatment_type": spec[1]["type"],
        "treatment": spec[1]["label"],
        "outcome_type": spec[2]["type"],
        "outcome": spec[2]["label"],
        "causal_model": spec[3]["label"],
        "estimator": spec[4]["label"],
        "estimated_effect": (causal_model, identified_estimand, estimate),
        "time": time.time() - start_time,
    }
    logging.info(f"Estimated effect: {result_dict}")
    return result_dict


def __estimate_effect(
    population_spec: Dict,
    treatment_spec: Dict,
    outcome_spec: List,
    model_spec: Dict,
    estimator_spec: Dict,
    context: Dict,
) -> Tuple[float, Dict]:
    """
    Estimate causal effect for a single analysis specification
    """
    # filter training data to create population
    all_data = context[population_spec["dataframe"]]
    population_id = population_spec.get("population_id")
    if population_id is None:
        population_data = all_data
    else:
        population_data = all_data[all_data[population_id] == 1]

    treatment_var = treatment_spec["variable"]
    population_data[treatment_var] = population_data[treatment_var].astype(bool)
    outcome_var = outcome_spec["variable"]
    causal_graph = copy.deepcopy(model_spec["causal_graph"])
    logging.info(
        f"population: {population_spec['label']}, "
        f"population_size: {population_data.shape[0]}, "
        f"treatment: {treatment_var}, outcome: {outcome_var}, "
        f"causal model: {model_spec['label']}, "
        f"estimator: {estimator_spec['method_name']}"
    )

    causal_model = dowhy.CausalModel(
        data=population_data,
        treatment=treatment_var,
        outcome=outcome_var,
        graph=causal_graph.replace("\n", " "),
        identify_vars=True,
    )

    # identify estimands and check if estimate using backdoor criterion is possible
    identified_estimand = causal_model.identify_effect(proceed_when_unidentifiable=True)
    if identified_estimand.estimands["backdoor"] is not None:
        try:
            estimator = CausalEstimator()

            # tune first stage models
            if (
                "propensity" in estimator_spec["method_name"]
                or "econml" in estimator_spec["method_name"]
            ):
                estimator.model_classifier = estimator.tune_classifier_model(
                    identified_estimand, causal_model
                )
            if "dml" in estimator_spec["method_name"]:
                estimator.model_regressor = estimator.tune_dml_regressor_model(
                    identified_estimand, causal_model
                )
            elif "dr" in estimator_spec["method_name"]:
                estimator.model_regressor = estimator.tune_dr_regressor_model(
                    identified_estimand, causal_model
                )
            estimator_config = estimator.config_estimator(copy.deepcopy(estimator_spec))
            logging.info(f"Estimator config: {estimator_config}")

            estimate = causal_model.estimate_effect(
                identified_estimand,
                method_name=estimator_config["method_name"],
                method_params=estimator_config["method_params"],
            )
            return causal_model, identified_estimand, estimate, population_data.shape[0]
        except Exception as e:
            logging.info(f"Cannot compute estimate: {e}. Returning None values")
            return None, None, None, population_data.shape[0]
    else:
        logging.info("Backdoor estimate is not possible")
        return None, None, None, population_data.shape[0]


def join_results(results: List) -> pd.DataFrame:
    """
    Join estimate, confidence interval, refutation and SHAP results
    """
    results_df = pd.DataFrame.from_dict(results)
    results_df = results_df[~results_df["estimated_effect"].isnull()]
    results_df = results_df.sort_values(by="estimated_effect")

    columns = [
        column
        for column in results_df.columns
        if not str(column).startswith("refuter")
        and not str(column).startswith("estimated")
        and str(column) != "time"
        and str(column) != "lower_bound"
        and str(column) != "upper_bound"
        and str(column) != "refutation_results"
    ]

    available_refuters = {
        column: np.sum
        for column in results_df.columns
        if column.startswith("refuter") or column.endswith("bound")
    }

    for column in results_df.columns:
        if column not in columns:
            results_df[column] = results_df[column].fillna(0)

    results_df = results_df.groupby(columns, sort=False).agg(
        {"estimated_effect": np.mean, "time": np.mean, **available_refuters}
    )

    # calculate final refutation result
    results_df["refutation_result"] = results_df.apply(
        lambda result: check_refutation_result(result), axis=1
    )

    results_df = results_df.reset_index().sort_values(by="estimated_effect")

    results_df.index = np.arange(1, len(results_df) + 1)
    results_df.index = results_df.index.set_names(["SpecificationID"])

    return results_df


def check_refutation_result(result: dict) -> int:
    """
    Helper function to output a single value representing
    refutation results for a given specification
    """
    if INCLUDE_SENSITIVITY_REFUTERS:
        refuters = set(
            [
                f'refuter_{refuter.replace("_refuter", "")}'
                for refuter in DEFAULT_REFUTATION_TESTS + SENSITIVITY_REFUTERS
            ]
        )
        sensitivity_refuters = set(
            [
                f'refuter_{refuter.replace("_refuter", "")}'
                for refuter in SENSITIVITY_REFUTERS
            ]
        )
        if not refuters.issubset(result.keys()):
            return -1
        else:
            fail_sensitivity_test = False
            for refuter in refuters:
                if result[refuter] != 1:
                    if refuter not in sensitivity_refuters:
                        return 0
                    else:
                        fail_sensitivity_test = True
            if fail_sensitivity_test:
                return 1
            else:
                return 2
    else:
        refuters = set(
            [
                f'refuter_{refuter.replace("_refuter", "")}'
                for refuter in DEFAULT_REFUTATION_TESTS
            ]
        )
        if not refuters.issubset(result.keys()):
            return -1
        else:
            for refuter in refuters:
                if result[refuter] != 1:
                    return 0
            return 2


In [ ]:
#
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE file in the project.
#

import logging

from typing import Dict, Tuple

from dowhy.causal_estimator import CausalEstimate

from showwhy_inference.inference_config import DEFAULT_CI_SIMULATIONS


class ConfidenceInterval:
    """
    Calculate confidence intervals using either default econML method
    or bootstrap resampling
    """

    def __init__(
        self,
        confidence_level: float = 0.95,
        num_simulations: int = DEFAULT_CI_SIMULATIONS,
        sample_size_fraction: float = 1.0,
    ):
        self.confidence_level = confidence_level
        self.num_simulations = num_simulations
        self.sample_size_fraction = sample_size_fraction

    def estimate_confidence_intervals(self, estimate: CausalEstimate) -> Dict:
        try:
            if "econml" in str(estimate.params["estimator_class"]):
                confidence_intervals = self.estimate_econml_confidence_intervals(
                    estimate
                )
            else:
                confidence_intervals = estimate.get_confidence_intervals(
                    confidence_level=self.confidence_level,
                    method="bootstrap",
                    num_simulations=self.num_simulations,
                    sample_size_fraction=self.sample_size_fraction,
                )
            return {
                "lower_bound": confidence_intervals[0],
                "upper_bound": confidence_intervals[1],
            }
        except Exception as e:
            logging.info(
                f"Cannot compute confidence interval: {e}."
                "Returning None values for confidence intervals"
            )
            return {"lower_bound": None, "upper_bound": None}

    def estimate_econml_confidence_intervals(
        self, estimate: CausalEstimate
    ) -> Tuple[float, float]:
        effect_modifiers = estimate.estimator._effect_modifiers
        return (
            estimate.estimator.estimator.effect_inference(X=effect_modifiers)
            .population_summary()
            .conf_int_mean()
        )


In [ ]:
population_specs = [{'type': 'Primary', 'label': 'Confirmed COVID-19 cases', 'dataframe': 'output', 'population_id': 'subject_inclusion'}]
treatment_specs = [{'type': 'Primary', 'label': 'Experienced chronic disease', 'variable': 'chronic_disease'}, {'type': 'Secondary', 'label': 'Experienced diabetes & hypertension chronic disease', 'variable': 'diabetes_hypertension'}]
outcome_specs = [{'type': 'Primary', 'label': 'Mortality', 'variable': 'death'}]
model_specs = [{'type': 'Maximum Model', 'label': 'Maximum Model', 'confounders': ['age_group', 'gender'], 'outcome_determinants': []}, {'type': 'Minimum Model', 'label': 'Minimum Model', 'confounders': ['age_group'], 'outcome_determinants': []}, {'type': 'Unadjusted Model', 'label': 'Unadjusted Model', 'confounders': [], 'outcome_determinants': []}]
estimator_specs = [{'type': 'Treatment Assignment Model', 'label': 'Inverse Propensity Weighting', 'require_propensity_score': True, 'method_name': 'backdoor.propensity_score_weighting'}, {'type': 'Treatment Assignment Model', 'label': 'Propensity Score Stratification', 'require_propensity_score': True, 'method_name': 'backdoor.propensity_score_stratification'}, {'type': 'Outcome Model', 'label': 'Linear Double Machine Learning', 'require_propensity_score': True, 'method_name': 'backdoor.econml.dml.LinearDML'}, {'type': 'Outcome Model', 'label': 'Linear Doubly Robust Learner', 'require_propensity_score': True, 'method_name': 'backdoor.econml.dr.LinearDRLearner'}, {'type': 'Outcome Model', 'label': 'Linear Regression', 'require_propensity_score': False, 'method_name': 'backdoor.linear_regression'}]
refuter_specs = {'num_simulations': 10}

In [ ]:
context = {}
context["output"] = output

# Calculate estimated effects

In [ ]:
specs = generate_all_specs(population_specs, treatment_specs, outcome_specs, model_specs, estimator_specs)
results = []
for spec in specs:
    results.append(estimate_specification(spec, context))

# Calculate Confidence Intervals

In [ ]:
confidence_interval_results = []
for previous_result in results:
   _, _, estimate = previous_result['estimated_effect']
   confidence_interval_result = ConfidenceInterval().estimate_confidence_intervals(estimate)
   confidence_interval_results.append({**previous_result, **confidence_interval_result, 'estimated_effect': estimate.value})

# Perform Refutation Tests

In [ ]:
def create_refuter_specs(num_simulations: int) -> List:
   refuter_specs = []
   for test in DEFAULT_REFUTATION_TESTS:
       spec = {
                 'method_name': test,
                 'num_simulations': num_simulations
              }
       refuter_specs.append(spec)
   return refuter_specs

In [ ]:
refuter_specs = create_refuter_specs(refuter_specs['num_simulations'])
specs = itertools.product(refuter_specs, results)
refuter_results = []
for refuter_spec, previous_result in specs:
    causal_model, identified_estimand, estimate = previous_result['estimated_effect']
    refuter_result = Refutation(refuter_spec).refute_estimate(causal_model, identified_estimand, estimate)
    refuter_results.append({**previous_result, **refuter_result, 'estimated_effect': estimate.value})

# Create Dataframe from results

In [ ]:
final_results = confidence_interval_results + refuter_results
results_df = join_results(final_results)
results_df.head()

# SHAP Analysis

In [ ]:
results_df = run_interpreter(results_df)
results_df.head()